# Real-Time Scheduling Tutorial: Rate Monotonic and Earliest Deadline First

## Introduction
Welcome to this Jupyter Notebook tutorial on **Real-Time Scheduling**! As an aspiring scientist, you're diving into a critical field used in systems like spacecraft, medical devices, and autonomous vehicles, where timing is everything. This notebook is your one-stop resource to master **Rate Monotonic Scheduling (RMS)** and **Earliest Deadline First (EDF)**, covering:
- **Theory**: Clear explanations with analogies.
- **Practical Code**: Python simulations to see scheduling in action.
- **Visualizations**: Timelines and utilization charts.
- **Applications**: Real-world use cases.
- **Research Directions**: Cutting-edge topics for your scientific career.
- **Rare Insights**: Lesser-known nuances for deeper understanding.
- **Mini-Project**: Simulate a basic real-time system.
- **Major Project**: Design a complex system with fault tolerance.
- **Additional Topics**: Concepts not covered in the previous tutorial, like aperiodic tasks, fault tolerance, and multiprocessor scheduling.

This notebook assumes you're a beginner, so we'll start from scratch, use simple language, and include everything you need to become a researcher in real-time systems.

## Prerequisites
- Install Python and the following libraries:
  ```bash
  pip install numpy matplotlib
  ```
- Basic Python knowledge (loops, functions, lists).
- No prior scheduling knowledge required!

## What is Real-Time Scheduling?
Real-time scheduling ensures tasks in a system meet strict timing deadlines. Imagine you're a chef in a kitchen (CPU) juggling dishes (tasks) that must be served on time (deadlines). Missing a deadline in a **hard real-time** system (e.g., an airbag deploying) is catastrophic, while in a **soft real-time** system (e.g., video streaming), it causes minor issues.

**Key Terms**:
- **Task**: A job with computation time (C) and period (T).
- **Period (T)**: Time between task activations.
- **Execution Time (C)**: Time to complete a task.
- **Deadline**: When a task must finish (often equals period in RMS).
- **Utilization (U)**: CPU usage, $ U = \frac{C}{T} $.

Let's explore RMS and EDF, then build simulations and projects!

## 1. Rate Monotonic Scheduling (RMS)

### Theory
**RMS** is a static priority scheduling algorithm for periodic tasks in hard real-time systems. Tasks with shorter periods (higher frequency) get higher priority. It's like prioritizing customers who order food more frequently in a kitchen.

**How It Works**:
1. Assign priorities based on period (shorter period = higher priority).
2. Preempt lower-priority tasks when a higher-priority task arrives.
3. Priorities are fixed.

**Schedulability Test**:
For $ n $ tasks, total utilization is:
$$ U = \sum_{i=1}^n \frac{C_i}{T_i} $$
Tasks are schedulable if:
$$ U \leq n(2^{1/n} - 1) $$
For example, for 3 tasks, $ U \leq 0.78 $.

**Analogy**: A teacher grades frequent assignments first to give timely feedback.

### Practical Code: RMS Simulation
Let's simulate RMS for three tasks:
- Task 1: $ C_1 = 1 $, $ T_1 = 4 $
- Task 2: $ C_2 = 1 $, $ T_2 = 5 $
- Task 3: $ C_3 = 2 $, $ T_3 = 10 $

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define tasks: (execution time, period, name)
tasks = [(1, 4, 'T1'), (1, 5, 'T2'), (2, 10, 'T3')]

# Calculate utilization
utilization = sum(c / t for c, t, _ in tasks)
n = len(tasks)
bound = n * (2 ** (1 / n) - 1)
print(f"Total Utilization: {utilization:.2f}")
print(f"RMS Bound for {n} tasks: {bound:.2f}")
print("Schedulable" if utilization <= bound else "May not be schedulable (needs further analysis)")

# Simulate RMS
def simulate_rms(tasks, duration):
    schedule = []
    current_time = 0
    # Track task instances: (task_index, remaining_time, deadline, name)
    active_tasks = []
    # Track next release time for each task
    next_release = [0] * len(tasks)
    
    while current_time < duration:
        # Add new task instances at their release times
        for i, (c, t, name) in enumerate(tasks):
            if current_time >= next_release[i]:
                active_tasks.append((i, c, next_release[i] + t, name))
                next_release[i] += t
        
        # Sort by priority (shorter period = higher priority)
        active_tasks.sort(key=lambda x: tasks[x[0]][1])
        
        # Execute highest priority task
        if active_tasks:
            task_idx, rem_time, deadline, name = active_tasks[0]
            schedule.append((current_time, current_time + 1, name))
            active_tasks[0] = (task_idx, rem_time - 1, deadline, name)
            
            # Remove completed tasks or tasks past deadline
            active_tasks = [t for t in active_tasks if t[1] > 0 and t[2] > current_time]
        else:
            schedule.append((current_time, current_time + 1, 'Idle'))
        
        current_time += 1
    
    return schedule

# Run simulation for 20 time units
schedule = simulate_rms(tasks, 20)

# Visualize schedule
plt.figure(figsize=(12, 3))
for start, end, name in schedule:
    color = {'T1': 'green', 'T2': 'blue', 'T3': 'orange', 'Idle': 'grey'}[name]
    plt.fill_between([start, end], 0, 1, color=color, alpha=0.5, label=name if start == 0 else None)
plt.title('RMS Schedule')
plt.xlabel('Time')
plt.yticks([])
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())
plt.grid(True)
plt.show()

### Explanation of Code
- **Tasks**: Defined as tuples (execution time, period, name).
- **Utilization**: Checks if tasks are schedulable using the RMS bound.
- **Simulation**: Tracks task releases and executes the highest-priority task (shortest period) at each time step.
- **Visualization**: Shows a timeline where each task is color-coded.

### Applications
- **Avionics**: RMS is used in flight control systems (e.g., Boeing 787), prioritizing tasks like altitude checks over less frequent tasks like cabin updates. [Source](https://www.sciencedirect.com/topics/computer-science/rate-monotonic-scheduling)
- **Automotive**: Engine control units use RMS for periodic sensor checks.

### Rare Insights
- **Data Dependencies**: RMS assumes no dependencies between tasks. If Task A must finish before Task B starts, RMS may waste CPU time unless dependencies are modeled. [Source](https://www.sciencedirect.com/topics/computer-science/rate-monotonic-scheduling)
- **Harmonic Periods**: If task periods are multiples of each other (e.g., 4, 8, 16), RMS can achieve higher utilization, sometimes up to 100%.

## 2. Earliest Deadline First (EDF)

### Theory
**EDF** is a dynamic priority scheduling algorithm where the task with the closest deadline gets the highest priority. Priorities change over time, making it flexible for both periodic and aperiodic tasks.

**How It Works**:
1. At each time step, pick the task with the earliest deadline.
2. Preempt if a task with an earlier deadline arrives.
3. Optimal for single processors (schedules any feasible task set).

**Schedulability Test**:
$$ U = \sum_{i=1}^n \frac{C_i}{T_i} \leq 1 $$
If $ U \leq 1 $, tasks are schedulable.

**Analogy**: In an emergency room, treat the patient with the most urgent condition first, regardless of arrival time.

### Practical Code: EDF Simulation
Using the same tasks as RMS.

In [ ]:
# Simulate EDF
def simulate_edf(tasks, duration):
    schedule = []
    current_time = 0
    active_tasks = []
    next_release = [0] * len(tasks)
    
    while current_time < duration:
        # Add new task instances
        for i, (c, t, name) in enumerate(tasks):
            if current_time >= next_release[i]:
                active_tasks.append((i, c, next_release[i] + t, name))
                next_release[i] += t
        
        # Sort by deadline (earliest deadline first)
        active_tasks.sort(key=lambda x: x[2])
        
        # Execute task with earliest deadline
        if active_tasks:
            task_idx, rem_time, deadline, name = active_tasks[0]
            schedule.append((current_time, current_time + 1, name))
            active_tasks[0] = (task_idx, rem_time - 1, deadline, name)
            active_tasks = [t for t in active_tasks if t[1] > 0 and t[2] > current_time]
        else:
            schedule.append((current_time, current_time + 1, 'Idle'))
        
        current_time += 1
    
    return schedule

# Run EDF simulation
schedule = simulate_edf(tasks, 20)

# Visualize
plt.figure(figsize=(12, 3))
for start, end, name in schedule:
    color = {'T1': 'green', 'T2': 'blue', 'T3': 'orange', 'Idle': 'grey'}[name]
    plt.fill_between([start, end], 0, 1, color=color, alpha=0.5, label=name if start == 0 else None)
plt.title('EDF Schedule')
plt.xlabel('Time')
plt.yticks([])
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())
plt.grid(True)
plt.show()

### Explanation of Code
- **Sorting**: Tasks are sorted by deadline at each time step.
- **Preemption**: The task with the earliest deadline runs, interrupting others if needed.
- **Visualization**: Shows how EDF prioritizes tasks differently from RMS.

### Applications
- **Multimedia**: EDF schedules video frame encoding in real-time streaming apps.
- **Robotics**: Handles dynamic tasks like obstacle avoidance in robots.

### Rare Insights
- **Domino Effect**: In overload conditions ($ U > 1 $), EDF can cause multiple tasks to miss deadlines, unlike RMS, which prioritizes high-frequency tasks. [Source](https://github.com/marciamart/Real-time-scheduling-Algorithm)
- **Aperiodic Tasks**: EDF naturally handles aperiodic tasks by prioritizing based on deadlines, unlike RMS, which requires extensions like sporadic servers.

## 3. Visualization: Utilization Comparison
Let’s visualize the utilization of our tasks.

In [ ]:
# Utilization bar chart
utilizations = [c / t for c, t, _ in tasks]
plt.figure(figsize=(8, 4))
plt.bar([name for _, _, name in tasks], utilizations, color=['green', 'blue', 'orange'])
plt.axhline(y=bound, color='red', linestyle='--', label=f'RMS Bound ({bound:.2f})')
plt.axhline(y=1, color='purple', linestyle='--', label='EDF Bound (1.0)')
plt.title('Task Utilization')
plt.ylabel('Utilization (C/T)')
plt.legend()
plt.grid(True)
plt.show()

## 4. Additional Topics for Scientists
These topics weren’t covered in the previous tutorial but are essential for research:

### Aperiodic Tasks
- **Definition**: Tasks without fixed periods (e.g., handling an emergency interrupt).
- **RMS Handling**: Use a **sporadic server** to allocate CPU time for aperiodic tasks without disrupting periodic ones.
- **EDF Advantage**: Naturally handles aperiodic tasks by assigning priorities based on deadlines.

### Fault Tolerance
- **Need**: Real-time systems (e.g., satellites) must handle failures like missed deadlines or hardware faults.
- **Techniques**:
  - **Redundancy**: Run critical tasks on multiple processors.
  - **Graceful Degradation**: Prioritize critical tasks in overload conditions.
- **EDF Issue**: In overload, EDF may fail multiple tasks, requiring recovery mechanisms.

### Multiprocessor Scheduling
- **Challenge**: RMS and EDF are designed for single processors. Multiprocessor systems require partitioning tasks or global scheduling.
- **Approaches**:
  - **Partitioned EDF**: Assign tasks to specific processors.
  - **Global EDF**: Allow tasks to migrate across processors.

### Context Switching Overheads
- **Reality**: Switching tasks takes time (not assumed in basic RMS/EDF models).
- **Impact**: Reduces effective CPU time, requiring adjusted schedulability tests.

## 5. Research Directions
As a scientist, explore these areas:
- **Mixed-Criticality Systems**: Schedule tasks with different criticality levels (e.g., safety-critical vs. non-critical).
- **Energy-Aware Scheduling**: Optimize scheduling for low-power devices like IoT sensors.
- **Machine Learning Integration**: Use ML to predict task arrivals and optimize EDF dynamically.
- **Multiprocessor Real-Time**: Develop scalable algorithms for multi-core systems.
- **Fault-Tolerant Scheduling**: Design systems that recover from missed deadlines or hardware failures.

## 6. Mini-Project: Simulate a Drone System
**Goal**: Simulate a drone’s real-time system with RMS and EDF.

**Tasks**:
- Navigation: $ C=2 $, $ T=10 $
- Camera: $ C=1 $, $ T=15 $
- Battery Check: $ C=3 $, $ T=20 $

**Code**: Modify the RMS/EDF code above to simulate these tasks over 30 time units. Add an aperiodic task (e.g., obstacle detection, $ C=2 $, arrives at $ t=12 $) for EDF.

In [ ]:
# Mini-Project: Drone System with EDF and Aperiodic Task
drone_tasks = [(2, 10, 'Navigation'), (1, 15, 'Camera'), (3, 20, 'Battery')]

# Add aperiodic task for EDF: (execution time, deadline, name, arrival time)
aperiodic_task = (2, 14, 'Obstacle', 12)

# EDF with aperiodic task
def simulate_edf_with_aperiodic(tasks, aperiodic, duration):
    schedule = []
    current_time = 0
    active_tasks = []
    next_release = [0] * len(tasks)
    aperiodic_added = False
    
    while current_time < duration:
        # Add periodic tasks
        for i, (c, t, name) in enumerate(tasks):
            if current_time >= next_release[i]:
                active_tasks.append((i, c, next_release[i] + t, name))
                next_release[i] += t
        
        # Add aperiodic task
        if not aperiodic_added and current_time >= aperiodic[3]:
            active_tasks.append((-1, aperiodic[0], aperiodic[1], aperiodic[2]))
            aperiodic_added = True
        
        # Sort by deadline
        active_tasks.sort(key=lambda x: x[2])
        
        if active_tasks:
            task_idx, rem_time, deadline, name = active_tasks[0]
            schedule.append((current_time, current_time + 1, name))
            active_tasks[0] = (task_idx, rem_time - 1, deadline, name)
            active_tasks = [t for t in active_tasks if t[1] > 0 and t[2] > current_time]
        else:
            schedule.append((current_time, current_time + 1, 'Idle'))
        
        current_time += 1
    
    return schedule

# Run and visualize
schedule = simulate_edf_with_aperiodic(drone_tasks, aperiodic_task, 30)
plt.figure(figsize=(12, 3))
for start, end, name in schedule:
    color = {'Navigation': 'green', 'Camera': 'blue', 'Battery': 'orange', 'Obstacle': 'red', 'Idle': 'grey'}[name]
    plt.fill_between([start, end], 0, 1, color=color, alpha=0.5, label=name if start == 0 else None)
plt.title('Drone EDF Schedule with Aperiodic Task')
plt.xlabel('Time')
plt.yticks([])
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())
plt.grid(True)
plt.show()

## 7. Major Project: Fault-Tolerant Satellite System
**Goal**: Design a satellite system with fault tolerance using EDF.

**Tasks**:
- Orbit Control: $ C=3 $, $ T=50 $
- Data Transmission: $ C=2 $, $ T=100 $
- Thermal Management: $ C=5 $, $ T=200 $

**Fault Tolerance**:
- Simulate a fault (e.g., Data Transmission misses deadline at $ t=120 $).
- Implement recovery: Reallocate CPU to critical tasks (Orbit Control).

**Code Outline**:
1. Simulate EDF with the tasks.
2. At $ t=120 $, increase $ C_2 $ to 4 (simulating fault).
3. Prioritize Orbit Control if utilization exceeds 1.

**Challenge**: Modify the EDF code to detect overload and skip non-critical tasks (e.g., Thermal Management).

In [ ]:
# Major Project: Fault-Tolerant Satellite System
satellite_tasks = [(3, 50, 'Orbit'), (2, 100, 'Data'), (5, 200, 'Thermal')]

def simulate_edf_fault_tolerant(tasks, duration, fault_time, fault_task_idx, fault_c):
    schedule = []
    current_time = 0
    active_tasks = []
    next_release = [0] * len(tasks)
    fault_applied = False
    
    while current_time < duration:
        # Apply fault at fault_time
        if current_time == fault_time and not fault_applied:
            tasks[fault_task_idx] = (fault_c, tasks[fault_task_idx][1], tasks[fault_task_idx][2])
            fault_applied = True
        
        # Add tasks
        for i, (c, t, name) in enumerate(tasks):
            if current_time >= next_release[i]:
                active_tasks.append((i, c, next_release[i] + t, name))
                next_release[i] += t
        
        # Check utilization for fault tolerance
        if sum(c / t for c, t, _ in tasks) > 1:
            # Prioritize critical task (Orbit)
            active_tasks = [t for t in active_tasks if t[3] == 'Orbit' or t[2] <= current_time + 10]
        
        # Sort by deadline
        active_tasks.sort(key=lambda x: x[2])
        
        if active_tasks:
            task_idx, rem_time, deadline, name = active_tasks[0]
            schedule.append((current_time, current_time + 1, name))
            active_tasks[0] = (task_idx, rem_time - 1, deadline, name)
            active_tasks = [t for t in active_tasks if t[1] > 0 and t[2] > current_time]
        else:
            schedule.append((current_time, current_time + 1, 'Idle'))
        
        current_time += 1
    
    return schedule

# Simulate with fault at t=120
schedule = simulate_edf_fault_tolerant(satellite_tasks, 200, 120, 1, 4)
plt.figure(figsize=(12, 3))
for start, end, name in schedule:
    color = {'Orbit': 'green', 'Data': 'blue', 'Thermal': 'orange', 'Idle': 'grey'}[name]
    plt.fill_between([start, end], 0, 1, color=color, alpha=0.5, label=name if start == 0 else None)
plt.title('Satellite EDF Schedule with Fault Tolerance')
plt.xlabel('Time')
plt.yticks([])
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())
plt.grid(True)
plt.show()

## 8. Research Insights and Applications

### Rare Insights
- **RMS Harmonic Periods**: When task periods are harmonic (e.g., 5, 10, 20), RMS can achieve near-100% utilization, a fact often overlooked in basic analyses. [Source](https://www.sciencedirect.com/topics/computer-science/rate-monotonic-scheduling)
- **EDF Overload Behavior**: EDF’s domino effect in overload can be mitigated with techniques like **bandwidth reservation**.
- **Context Switching**: Real systems incur overhead (0.1-1ms per switch), which must be factored into schedulability tests for accuracy.

### Applications
- **Medical Devices**: Pacemakers use RMS for periodic heart monitoring.
- **IoT**: EDF schedules sensor data processing in smart cities.
- **Space Exploration**: NASA’s Apollo missions used RMS for critical tasks. [Source](https://www.sciencedirect.com/topics/computer-science/rate-monotonic-scheduling)

## 9. Next Steps for Scientists
- **Experiment**: Extend the mini-project to include context-switching delays.
- **Read**: Liu and Layland’s 1973 paper on RMS for foundational theory.
- **Tools**: Use RTEMS or FreeRTOS to implement real-time scheduling.
- **Publish**: Simulate a novel scheduling algorithm combining RMS and EDF for mixed-criticality systems.

## 10. Conclusion
This notebook equips you with the theory, code, and projects to master RMS and EDF. You’ve learned how to simulate real-time systems, visualize schedules, and design fault-tolerant systems. As a scientist, use these skills to innovate in fields like robotics, aerospace, and IoT. Keep exploring, and you’re on your way to groundbreaking research!

**Note**: Save this notebook regularly (Ctrl+S) to preserve your work. For the mini and major projects, experiment with different task sets and analyze their schedulability.